<a href="https://colab.research.google.com/github/bye23mj/suanLee/blob/main/NL_13_%ED%8A%B8%EB%9E%9C%EC%8A%A4%ED%8F%AC%EB%A8%B8_(Transformer).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 트랜스포머 (Transformer)

* 참고: https://wikidocs.net/31379

* attention mechanism은 seq2seq의 입력 시퀀스 정보 손실을 보정해주기 위해 사용됨
* attention mechanism을 보정 목적이 아닌, 인코더와 디코더로 구성한 모델이 바로 트랜스포머
* 트랜스포머는 RNN을 사용하지 않고 인코더와 디코더를 설계하였으며, 성능도 RNN보다 우수함



## 포지셔널 인코딩

* 기존의 RNN은 단어의 위치를 따라 순차적으로 입력받아 단어의 위치정보를 활용할 수 있었음
* 트랜스포머의 경우, RNN을 활용하지 않았기 때문에 단어의 위치정보를 다른 방식으로 줄 필요가 있음
* 이를 위해 **각 단어의 임베딩 벡터에 위치 정보들을 더하게 되는데** 이를 포지셔널 인코딩이라 함
* 보통 포지셔널 인코딩은 sin, cos을 이용하여 계산

In [ ]:
import tensorflow as tf
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)

# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

strategy = tf.distribute.TPUStrategy(resolver)

All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


In [ ]:
import tensorflow as tf
import numpy as np

def Positional_encoding(dim, sentence_length):
  encoded_vec = np.array([pos / np.power(10000, 2 * i / dim) for pos in range(sentence_length) for i in range(dim)])
  encoded_vec[::2] = np.sin(encoded_vec[::2])
  encoded_vec[1::2] = np.cos(encoded_vec[1::2])
  return tf.constant(encoded_vec.reshape([sentence_length, dim]), dtype=tf.float32)

## 레이어 정규화

*  레이어 정규화에서는 텐서의 마지막 차원에 대해 평균과 분산을 구하고, 이 값을 통해 값을 정규화함
*  해당 정규화를 각 층의 연결에 편리하게 적용하기 위해 함수화한 `sublayer_connection()`을 선언

In [ ]:
def layer_norm(inputs, eps=1e-6):
  feature_shape = inputs.get_shape()[-1:]
  mean = tf.keras.backend.mean(inputs, [-1], keepdims=True)
  std = tf.keras.backend.std(inputs, [-1], keepdims=True)
  beta = tf.Variable(tf.zeros(feature_shape), trainable=False)
  gamma = tf.Variable(tf.ones(feature_shape), trainable=False)
  return gamma * (inputs - mean) / (std + eps) + beta

In [ ]:
def sublayer_connection(inputs, sublayer, dropout=0.2):
  outputs = layer_norm(inputs + tf.keras.layers.Dropout(dropout)(sublayer))
  return outputs

## 어텐션



*   트랜스포머 모델의 핵심이 되는 부분
*   트랜스포머에서는 multi-head attention과 self attention이라는 개념을 사용
  1.   multi-head attention
      * 디코더가 가지는 차원을 나누어 병렬로 어텐션을 진행
      *  마지막엔 병렬로 각 진행해 얻은 어텐션 헤드를 모두 연결
      * 이로 인해 다양한 시각에서 정보를 수집할 수 있는 효과를 얻음
  2.   self attention
      *   일반적인 어텐션의 경우, 특정 시점의 디코더 은닉상태와 모든 시점의 인코더 은닉상태를 활용
      *   이는 입력 문장과 다른 문장에 존재하는 단어간의 어텐션을 의미함
      *   반면 self attention은 은닉 상태를 동일하게 하여 어텐션을 진행
      *   이는 입력 문장 내 단어간의 어텐션을 의미함




*   트랜스포머 제안 논문에서는 scaled-dot product attention을 활용해 모델을 작성함



### scaled-dot product attention 구현

* scaled-dot product attention은 앞서 학습한 dot product attention과 거의 유사함
* 단 attention을 진행할 때 어텐션 스코어를 계산할 때 내적 값을 정규화
* 트랜스포머에서는 정규화할 때 K 벡터(=디코더 셀의 은닉 상태)의 차원을 루트를 취한 값을 사용

In [ ]:
def scaled_dot_product_attention(query, key, value, masked=False):
  key_dim_size = float(key.get_shape().as_list()[-1])
  key = tf.transpose(key, perm=[0, 2, 1])

  outputs = tf.matmul(query , key) / tf.sqrt(key_dim_size)

  if masked:
    diag_vals = tf.ones_like(outputs[0, :, :])
    tril = tf.linalg.LinearOperatorLowerTriangular(diag_vals).to_dense()
    masks = tf.tile(tf.expand_dims(tril, 0), [tf.shape(outputs)[0], 1, 1])
    paddings = tf.ones_like(masks)*(-2**30)
    outputs= tf.where(tf.equal(masks, 0), paddings, outputs)

  attension_map = tf.nn.softmax(outputs)
  return tf.matmul(attension_map, value)

### multi-head attention 구현

* multi-head attention의 구현 과정
  1. query, key, value에 해당하는 값을 받고, 해당 값에 해당하는 행렬 생성
  2. 생성된 행렬들을 heads에 해당하는 수만큼 분리
  3. 분리한 행렬들에 대해 각각 어텐션을 수행
  4. 각 어텐션 결과들을 연결해 최종 어텐션 결과 생성





In [ ]:
def multi_head_attention(query, key, value, num_units, heads, masked=False):
  query = tf.keras.layers.Dense(num_units, activation=tf.nn.relu)(query)
  key = tf.keras.layers.Dense(num_units, activation=tf.nn.relu)(key)
  value = tf.keras.layers.Dense(num_units, activation=tf.nn.relu)(value)

  query = tf.concat(tf.split(query, heads, axis=-1), axis=0)
  key = tf.concat(tf.split(key, heads, axis=-1), axis=0)
  value = tf.concat(tf.split(value, heads, axis=-1), axis=0)

  attention_map = scaled_dot_product_attention(query, key, value, masked)
  attn_outputs = tf.concat(tf.split(attention_map, heads, axis=0), axis=-1)
  attn_outputs = tf.keras.layers.Dense(num_units, activation=tf.nn.relu)(attn_outputs)

  return attn_outputs

## 포지션-와이즈 피드 포워드 신경망



*   multi-head attention의 결과인 행렬을 입력받아 연산
*   일반적인 완전 연결 신경망(Dense layer)를 사용
*   position-wise FFNN은 인코더와 디코더에 모두 존재



In [ ]:
def feed_forward(inputs, num_units):
  feature_shape = inputs.get_shape()[-1]
  inner_layer = tf.keras.layers.Dense(num_units, activation=tf.nn.relu)(inputs)
  outputs = tf.keras.layers.Dense(feature_shape)(inner_layer)
  return outputs

## 인코더


* 인코더는 하나의 어텐션을 사용
  + encoder self-attention (multi-head self-attention과 동일)

In [ ]:
def encoder_module(inputs, model_dim, ffn_dim, heads):
  self_attn = sublayer_connection(inputs, multi_head_attention(inputs, inputs, inputs, model_dim, heads))
  outputs = sublayer_connection(self_attn, feed_forward(self_attn, ffn_dim))
  return outputs

def encoder(inputs, model_dim, ffn_dim, heads, num_layers):
  outputs = inputs
  for i in range(num_layers):
    outputs = encoder_module(outputs, model_dim, ffn_dim, heads)

  return outputs

## 디코더

* 디코더는 다음과 같은 구성의 반복으로 이루어짐
  1. masked decoder self-attention
  2. encoder-decoder attention
  3. position-wise FFNN

* 디코더에서는 2종류의 어텐션을 사용
  1.   masked decoder self-attention
    *   디코더에서는 인코더와는 달리 순차적으로 결과를 만들어 내야하기 때문에 다른 어텐션 방법을 사용함
    *   디코더 예측 시점 이후의 위치에 attention을 할 수 없도록 masking 처리
    *   결국 예측 시점에서 예측은 미리 알고 있는 위치까지만의 결과에 의존
  2.   encoder-decoder attention
    *   앞서 설명한 multi-head attention과 동일



In [ ]:
def decoder_module(inputs, encoder_outputs, model_dim, ffn_dim, heads):
  masked_self_attn = sublayer_connection(inputs, 
                                         multi_head_attention(inputs, inputs, inputs, model_dim, heads, masked=True))
  self_attn = sublayer_connection(masked_self_attn, 
                                         multi_head_attention(masked_self_attn, encoder_outputs, encoder_outputs, model_dim, heads, masked=True))
  outputs = sublayer_connection(self_attn, feed_forward(self_attn, ffn_dim))
  return outputs

def decoder(inputs, encoders_outputs, model_dim, ffn_dim, heads, num_layers):
  outputs = inputs
  for i in range(num_layers):
    outputs = decoder_module(outputs, encoders_outputs, model_dim, ffn_dim, heads)

  return outputs

## 트랜스포머를 활용한 챗봇

### konlpy 라이브러리

*    한글을 처리하기 위해 konlpy 라이브러리 설치

In [ ]:
from konlpy.tag import Twitter
import pandas as pd
import tensorflow as tf
import enum
import os
import re
from sklearn.model_selection import train_test_split
import numpy as np
#from configs import DEFINES

from tqdm import tqdm

In [ ]:
!pip install configs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 5.0 MB/s 
     |████████████████████████████████| 453 kB 37.3 MB/s 


### 데이터 준비

* 처리에 필요한 각종 변수 선언
* filters에 해당되는 문자를 걸러주는 정규 표현식 컴파일



In [ ]:
import re
import tensorflow as tf

filters = "([~.,!?\"':;)(])"
PAD = '<PADDING>'
STD = '<START>'
END = '<END>'
UNK = '<UNKNOWN>'

PAD_INDEX = 0
STD_INDEX = 1
END_INDEX = 2
UNK_INDEX = 3

MARKER = [PAD, STD, END, UNK]
CHANGE_FILTER = re.compile(filters)

* 주소에서 데이터를 가져오는 `load_data()` 함수 선언



In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

def load_data(data_path):
  data_df = pd.read_csv(data_path, header=0)
  question, answer = list(data_df['Q']), list(data_df['A'])
  train_inputs, eval_inputs, train_label, eval_label = train_test_split(question, answer,
                                                                       test_size=0.33, random_state=111)
  return train_inputs, train_label, eval_inputs, eval_label

* 처리에 필요한 단어 사전을 생성하는 `load_vocab()` 함수 선언

In [ ]:
def load_vocabulary(data_path):
  data_df = pd.read_csv(data_path, encoding='utf-8')
  question, answer = list(data_df['Q']), list(data_df['A'])
  if tokenize_as_morph:
    question = prepro_like_morphlized(question)
    answer = prepro_like_morphlized(answer)

  data = []
  data.extend(question)
  data.extend(answer)
  words = data_tokenizer(data)
  words = list(set(words))
  words[:0] = MARKER

  char2idx = {char:idx for idx, char in enumerate(words)}
  idx2char = {idx:char for idx, char in enumerate(words)}
  return char2idx, idx2char, len(char2idx)

* 문자열 데이터를 학습에 사용될 수 있도록 변현하는 `prepro_like_morphlized()` 함수 선언



In [ ]:
from konlpy.tag import Okt

def prepro_like_morphlized(data):
    # 형태소 분석 모듈 객체를 생성합니다.

    morph_analyzer = Okt()
    # 형태소 토크나이즈 결과 문장을 받을 리스트를 생성합니다.
    result_data = list()
    # 데이터에 있는 매 문장에 대해 토크나이즈를 할 수 있도록 반복문을 선언합니다.
    for seq in tqdm(data):
        # Okt.morphs 함수를 통해 토크나이즈 된 리스트 객체를 받고 다시 공백문자를 기준으로 하여 문자열로 재구성 해줍니다.
        morphlized_seq = " ".join(morph_analyzer.morphs(seq.replace(' ', '')))
        result_data.append(morphlized_seq)

    return result_data

* 단어 사전을 만들기 위해 단어들을 분리하는 `data_tokenizer()` 함수 선언

In [ ]:
def data_tokenizer(data):
    # 토크나이징 해서 담을 배열 생성
    words = []
    for sentence in data:
        # FILTERS = "([~.,!?\"':;)(])"
        # 위 필터와 같은 값들을 정규화 표현식을 통해서 모두 "" 으로 변환 해주는 부분이다.
        sentence = re.sub(CHANGE_FILTER, "", sentence)
        for word in sentence.split():
            words.append(word)
    # 토그나이징과 정규표현식을 통해 만들어진 값들을 넘겨 준다.
    return [word for word in words if word]

* encoder의 입력을 구성하기 위한 함수 `enc_processing()` 선언



In [ ]:
# 인덱스화 할 value와 키가 워드이고 값이 인덱스인 딕셔너리를 받는다.
def enc_processing(value, dictionary):
    # 인덱스 값들을 가지고 있는 배열이다.(누적된다.)
    sequences_input_index = []
    # 하나의 인코딩 되는 문장의 길이를 가지고 있다.(누적된다.)
    sequences_length = []
    # 형태소 토크나이징 사용 유무
    if tokenize_as_morph:
        value = prepro_like_morphlized(value)

    # 한줄씩 불어온다.
    for sequence in value:
        # FILTERS = "([~.,!?\"':;)(])"
        # 정규화를 사용하여 필터에 들어 있는 값들을 "" 으로 치환 한다.
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        # 하나의 문장을 인코딩 할때 가지고 있기 위한 배열이다.
        sequence_index = []
        # 문장을 스페이스 단위로 자르고 있다.
        for word in sequence.split():
            # 잘려진 단어들이 딕셔너리에 존재 하는지 보고 그 값을 가져와 sequence_index에 추가한다.
            if dictionary.get(word) is not None:
                sequence_index.extend([dictionary[word]])
            # 잘려진 단어가 딕셔너리에 존재 하지 않는 경우 이므로 UNK(2)를 넣어 준다.
            else:
                sequence_index.extend([dictionary[UNK]])
        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 자르고 있다.
        if len(sequence_index) > max_len:
            sequence_index = sequence_index[:max_len]
        # 하나의 문장에 길이를 넣어주고 있다.
        sequences_length.append(len(sequence_index))
        # max_sequence_length보다 문장 길이가 
        # 작다면 빈 부분에 PAD(0)를 넣어준다.
        sequence_index += (max_len - len(sequence_index)) * [dictionary[PAD]]
        # 인덱스화 되어 있는 값을 
        # sequences_input_index에 넣어 준다.
        sequences_input_index.append(sequence_index)
    # 인덱스화된 일반 배열을 넘파이 배열로 변경한다. 
    # 이유는 텐서플로우 dataset에 넣어 주기 위한 사전 작업이다.
    # 넘파이 배열에 인덱스화된 배열과 그 길이를 넘겨준다.  
    return np.asarray(sequences_input_index), sequences_length

* decoder의 입력을 구성하기 위한 함수 `dec_output_processing()` 선언

In [ ]:
def dec_output_processing(value, dictionary):
    # 인덱스 값들을 가지고 있는 배열이다.(누적된다)
    sequences_output_index = []
    # 하나의 디코딩 입력 되는 문장의 길이를 가지고 있다.(누적된다)
    sequences_length = []
    # 형태소 토크나이징 사용 유무
    if tokenize_as_morph:
        value = prepro_like_morphlized(value)
    # 한줄씩 불어온다.
    for sequence in value:
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        sequence_index = []
        sequence_index = [dictionary[STD]] + [dictionary[word] for word in sequence.split()]
        if len(sequence_index) > max_len:
            sequence_index = sequence_index[:max_len]
        sequences_length.append(len(sequence_index))
        sequence_index += (max_len - len(sequence_index)) * [dictionary[PAD]]
        sequences_output_index.append(sequence_index)
    return np.asarray(sequences_output_index), sequences_length

* decoder의 출력을 구성하기 위한 함수 `dec_target_processing()` 선언

In [ ]:
def dec_target_processing(value, dictionary):
    # 인덱스 값들을 가지고 있는 배열이다.(누적된다)
    sequences_target_index = []
    # 형태소 토크나이징 사용 유무
    if tokenize_as_morph:
        value = prepro_like_morphlized(value)
    # 한줄씩 불어온다.
    for sequence in value:
        # FILTERS = "([~.,!?\"':;)(])"
        # 정규화를 사용하여 필터에 들어 있는 값들을 "" 으로 치환 한다.
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        # 문장에서 스페이스 단위별로 단어를 가져와서 
        # 딕셔너리의 값인 인덱스를 넣어 준다.
        # 디코딩 출력의 마지막에 END를 넣어 준다.
        sequence_index = [dictionary[word] for word in sequence.split()]
        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 자르고 있다.
        # 그리고 END 토큰을 넣어 준다
        if len(sequence_index) >= max_len:
            sequence_index = sequence_index[:max_len-1] + [dictionary[END]]
        else:
            sequence_index += [dictionary[END]]
        # max_sequence_length보다 문장 길이가 작다면 빈 부분에 PAD(0)를 넣어준다.
        sequence_index += (max_len - len(sequence_index)) * [dictionary[PAD]]
        # 인덱스화 되어 있는 값을 sequences_target_index에 넣어 준다.
        sequences_target_index.append(sequence_index)
    # 인덱스화된 일반 배열을 넘파이 배열로 변경한다. 
    # 이유는 텐서플로우 dataset에 넣어 주기 위한 사전 작업이다.
    # 넘파이 배열에 인덱스화된 배열과 그 길이를 넘겨준다.
    return np.asarray(sequences_target_index)

* 모델에 데이터를 효율적으로 투입하도록 `train_input_fn()`, `eval_input_fn()` 함수 선언
* `rearrange()`는 dataset 객체가 데이터를 어떻게 변형시킬지 정의해둔 함수
* dataset.map은 rearrange 함수를 기반으로 데이터를 변형



In [ ]:
# 학습에 들어가 배치 데이터를 만드는 함수이다.
def train_input_fn(train_input_enc, train_output_dec, train_target_dec, batch_size):
    # Dataset을 생성하는 부분으로써 from_tensor_slices부분은 각각 한 문장으로 자른다고 보면 된다.
    # train_input_enc, train_output_dec, train_target_dec 3개를 각각 한문장으로 나눈다.
    dataset = tf.compat.v1.data.Dataset.from_tensor_slices((train_input_enc, train_output_dec, train_target_dec))
    # 전체 데이터를 썩는다.
    dataset = dataset.shuffle(buffer_size=len(train_input_enc))
    # 배치 인자 값이 없다면  에러를 발생 시킨다.
    assert batch_size is not None, "train batchSize must not be None"
    # from_tensor_slices를 통해 나눈것을 배치크기 만큼 묶어 준다.
    dataset = dataset.batch(batch_size, drop_remainder=True)
    # 데이터 각 요소에 대해서 rearrange 함수를 통해서 요소를 변환하여 맵으로 구성한다.
    dataset = dataset.map(rearrange) 
    # repeat()함수에 원하는 에포크 수를 넣을수 있으면 아무 인자도 없다면 무한으로 이터레이터 된다.
    dataset = dataset.repeat()
    # make_one_shot_iterator를 통해 이터레이터를 만들어 준다.
    iterator = dataset.make_one_shot_iterator()
    # 이터레이터를 통해 다음 항목의 텐서 개체를 넘겨준다.
    return iterator.get_next()

# 평가에 들어가 배치 데이터를 만드는 함수이다.
def eval_input_fn(eval_input_enc, eval_output_dec, eval_target_dec, batch_size):
    # Dataset을 생성하는 부분으로써 from_tensor_slices부분은 각각 한 문장으로 자른다고 보면 된다.
    # eval_input_enc, eval_output_dec, eval_target_dec 3개를 각각 한문장으로 나눈다.
    dataset = tf.compat.v1.data.Dataset.from_tensor_slices((eval_input_enc, eval_output_dec, eval_target_dec))
    # 전체 데이터를 섞는다.
    dataset = dataset.shuffle(buffer_size=len(eval_input_enc))
    # 배치 인자 값이 없다면  에러를 발생 시킨다.
    assert batch_size is not None, "eval batchSize must not be None"
    # from_tensor_slices를 통해 나눈것을 배치크기 만큼 묶어 준다.
    dataset = dataset.batch(batch_size, drop_remainder=True)
    # 데이터 각 요소에 대해서 rearrange 함수를 통해서 요소를 변환하여 맵으로 구성한다.
    dataset = dataset.map(rearrange)
    # repeat()함수에 원하는 에포크 수를 넣을수 있으면 아무 인자도 없다면 무한으로 이터레이터 된다.
    # 평가이므로 1회만 동작 시킨다.
    dataset = dataset.repeat(1)
    # make_one_shot_iterator를 통해 이터레이터를 만들어 준다.
    iterator = dataset.make_one_shot_iterator()
    # 이터레이터를 통해 다음 항목의 텐서 개체를 넘겨준다.
    return iterator.get_next()  

def rearrange(input, output, target):
    features = {"input": input, "output": output}
    return features, target


* 모델의 예측은 배열로 생성되기 때문에 이를 확인하기 위해선 문자열로 변환이 필요
* 예측을 문자열로 변환해주는 `pred2string()` 함수 선언


In [ ]:
# 인덱스를 스트링으로 변경하는 함수이다.
# 바꾸고자 하는 인덱스 value와 인덱스를 키로 가지고 있고 값으로 단어를 가지고 있는 딕셔너리를 받는다.
def pred2string(value, dictionary):
    # 텍스트 문장을 보관할 배열을 선언한다.
    sentence_string = []
    is_finished = False

    # 인덱스 배열 하나를 꺼내서 v에 넘겨준다.
    for v in value:
        # 딕셔너리에 있는 단어로 변경해서 배열에 담는다.
        print(v['indexs'])
        #for index in v['indexs']:
        #    print(index)
        sentence_string = [dictionary[index] for index in v['indexs']]

    print("***********************")
    print(sentence_string)
    print("***********************")
    answer = ""
    # 패딩값도 담겨 있으므로 패딩은 모두 스페이스 처리 한다.
    for word in sentence_string:
        if word == END:
          is_finished = True
          break
        
        if word != PAD and word != END:
          answer += word
          answer += " "
    return answer, is_finished

def pred_next_string(value, dictionary):
    # 텍스트 문장을 보관할 배열을 선언한다.
    sentence_string = []
    # 인덱스 배열 하나를 꺼내서 v에 넘겨준다.
    for v in value:
        # 딕셔너리에 있는 단어로 변경해서 배열에 담는다.
        sentence_string = [dictionary[index] for index in v['indexs']]
    
    answer = ""
    # 패딩값도 담겨 있으므로 패딩은 모두 스페이스 처리 한다.
    for word in sentence_string:
        if word not in PAD and word not in END:
            answer += word
            answer += " "
    # 결과를 출력한다.
    return answer

* 챗봇 데이터 URL: https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv
* 데이터 주소에서 데이터를 읽어들여 단어 사전과 사용 데이터 구성

In [ ]:
import pandas as pd

tokenize_as_morph = True

data_path =  '/content/ChatBotData.csv'

char2idx, idx2char, len_vocab = load_vocabulary(data_path)
train_input, train_label, eval_input, eval_label = load_data(data_path)

100%|██████████| 11823/11823 [00:52<00:00, 225.87it/s]


In [ ]:
print(len(train_input))
print(len(train_label))
print(len(eval_input))
print(len(eval_label))

7921
7921
3902
3902


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from collections import Counter

from konlpy.tag import Okt

from functools import reduce
from wordcloud import WordCloud

DATA_IN_PATH =  '/content/'

data = pd.read_csv(DATA_IN_PATH + 'ChatBotData.csv', encoding='utf-8')

### 모델 구성

* 앞서 작성한 트랜스포머 모델을 결합해 학습에 사용할 모델을 구성함

In [ ]:
def model(features, labels, mode, params):
  TRAIN = mode == tf.estimator.ModeKeys.TRAIN
  EVAL = mode == tf.estimator.ModeKeys.EVAL
  PREDICT = mode == tf.estimator.ModeKeys.PREDICT

  position_encode = Positional_encoding(params['embedding_size'], params['max_len'])
  if params['xavier_initializer']:
    embeddings_initializer = 'glorot_normal'
  else:
    embeddings_initializer = 'uniform'

  embedding = tf.keras.layers.Embedding(params['len_vocab'],
                                        params['embedding_size'],
                                        embeddings_initializer=embeddings_initializer)
  
  x_embedded_matrix = embedding(features['input']) + position_encode
  y_embedded_matrix = embedding(features['output']) + position_encode

  encoder_outputs = encoder(x_embedded_matrix, params['model_hidden_size'], params['ffn_hidden_size'],
                           params['attention_head_size'], params['layer_size'])
  
  decoder_outputs = decoder(y_embedded_matrix, encoder_outputs, params['model_hidden_size'], 
                           params['ffn_hidden_size'], params['attention_head_size'], params['layer_size'])
  
  logits = tf.keras.layers.Dense(params['len_vocab'])(decoder_outputs)
  predict = tf.argmax(logits, 2)

  if PREDICT:
    predictions = {'indexs': predict,'logits': logits}
    return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  labels_ = tf.one_hot(labels, params['len_vocab'])
  loss = tf.reduce_mean(tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels_))

  accuracy = tf.compat.v1.metrics.accuracy(labels=labels, predictions=predict)
  #accuracy = tf.metrics.accuracy(labels=labels, predictions=predict, name='accOp')  

  metrics = {'accuracy':accuracy}
  tf.summary.scalar('accuracy', accuracy[1])

  if EVAL:
    return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)
  assert TRAIN

  optimizer = tf.compat.v1.train.AdadeltaOptimizer(learning_rate=params['learning_rate'])
  train_op = optimizer.minimize(loss, global_step=tf.compat.v1.train.get_global_step())
  return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)


### 모델 학습

*   필요한 각종 인자들을 설정
*   인자에 따라 학습 결과가 달라질 수 있기 때문에 세심한 조정이 필요


In [ ]:
max_len = 25
epoch = 5000
batch_size = 250
embedding_size = 100
model_hidden_size = 100
ffn_hidden_size = 100
attention_head_size = 100
lr = 0.001
layer_size = 3
xavier_initializer = True

*   앞서 선언한 processing 함수로 데이터를 모델에 투입할 수 있도록 가공
*   평가 데이터에도 동일하게 가공

In [ ]:
train_input_enc, train_input_enc_length = enc_processing(train_input, char2idx)
train_output_dec, train_output_enc_length = dec_output_processing(train_label, char2idx)
train_target_dec = dec_target_processing(train_label, char2idx)

eval_input_enc, eval_input_enc_length = enc_processing(eval_input, char2idx)
eval_output_dec, eval_output_enc_length = dec_output_processing(eval_input, char2idx)
eval_target_dec = dec_target_processing(eval_label, char2idx)

100%|██████████| 3902/3902 [00:17<00:00, 228.82it/s]


In [ ]:
print(train_input_enc.shape)
print(train_output_dec.shape)
print(train_target_dec.shape)
print(eval_input_enc.shape)
print(eval_output_dec.shape)
print(eval_target_dec.shape)

(7921, 25)
(7921, 25)
(7921, 25)
(3902, 25)
(3902, 25)
(7921, 25)


* 앞서 선언한 함수를 통해 모델을 선언하고 학습
* `tf.estimator`를 사용해 간편하게 학습 모듈 구성


In [ ]:
with strategy.scope():
  transformer = tf.estimator.Estimator(
      model_fn = model,
      params = {'embedding_size':embedding_size,
                'model_hidden_size':model_hidden_size,
                'ffn_hidden_size':ffn_hidden_size,
                'attention_head_size':attention_head_size,
                'learning_rate':lr,
                'len_vocab':len_vocab,
                'layer_size':layer_size,
                'max_len':max_len,
                'xavier_initializer':xavier_initializer}
  )

* 학습한 모델을 사용해 챗봇을 사용
* 예측 결과를 문자열로 변환할 때는 앞서 선언한 `pred2string()` 함수를 이용
* 입력에 대한 응답이 생성되는 것을 확인할 수 있음


In [ ]:
transformer.train(input_fn=lambda: train_input_fn(train_input_enc, train_output_dec, train_target_dec, batch_size), steps=epoch)

Instructions for updating:
Use standard file utilities to get mtimes.


In [ ]:
eval_result = transformer.evaluate(input_fn=lambda: eval_input_fn(eval_input_enc, eval_output_dec, eval_target_dec,  batch_size))

In [ ]:
print('\nEVAL set accuracy: {accuracy:0.3f}\n'.format(**eval_result))


EVAL set accuracy: 0.766



### 예측

* 학습한 모델을 사용해 챗봇을 사용
* 예측 결과를 문자열로 변환할 때는 앞서 선언한 `pred2string()` 함수를 이용
* 입력에 대한 응답이 생성되는 것을 확인할 수 있음


In [ ]:
def chatbot(sentence):
  pred_input_enc, pred_input_enc_length = enc_processing([sentence], char2idx)
  pred_output_dec, pred_output_dec_length = dec_output_processing([""], char2idx)
  pred_target_dec = dec_target_processing([""], char2idx)

  for i in range(max_len):
    if i > 0:
      pred_output_dec, pred_output_dec_length = dec_output_processing([answer], char2idx)
      pred_target_dec = dec_target_processing([answer], char2idx)

    predictions = transformer.predict(input_fn=lambda: eval_input_fn(pred_input_enc, pred_output_dec, pred_target_dec,  1))

    answer, finished = pred2string(predictions, idx2char)

    if finished:
      break

  return answer

In [ ]:
chatbot("안녕?")

100%|██████████| 1/1 [00:00<00:00, 3469.23it/s]


[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
***********************
['<END>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>']
***********************


''